<a href="https://www.kaggle.com/code/idiotprofessor/average-word2vec-and-lstm?scriptVersionId=207549510" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Library

In [ ]:
#Importing all the libraries to be used
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split   
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, classification_report, accuracy_score, f1_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB

# Reading Data

In [ ]:
data = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='ISO-8859-1')

In [ ]:
df = data.copy()

In [ ]:
df.head()

In [ ]:
df.drop(columns = ['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace = True)

In [ ]:
df.rename(columns = {'v1': 'output', 'v2':'Message'}, inplace = True)

In [ ]:
df.info()

In [ ]:
df['output'].value_counts()

# Visualizing data distribution

In [ ]:
plt.figure(figsize=(6,6))
sns.histplot(data = df, x= 'output', hue = 'output');

In [ ]:
df['output'].replace({'ham':0, 'spam':1})

In [ ]:
output_dummies = pd.get_dummies(df['output'], prefix='output')
df = pd.concat([df.drop(columns = ['output']), output_dummies], axis = 1)
df.head()

In [ ]:
df.replace({True : 1, False : 0}, inplace = True)

In [ ]:
df.drop(columns = ['output_ham'], inplace = True)

In [ ]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# Corpus after removing all the stop word and special char

In [ ]:
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['Message'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
len(corpus)

# Converting each words into vector, choosing most important 2500

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [ ]:
y=df.drop('Message', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Creating models

**1 Multinomial NB**

In [ ]:
spam_detect_model = MultinomialNB().fit(X_train, y_train)
y_pred = spam_detect_model.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
# Generating the classification report
report = classification_report(y_test, y_pred) 
print(report)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix');

**2. Random Forest**

In [ ]:
spam_detect_model1 = RandomForestClassifier().fit(X_train, y_train)
y_pred1 = spam_detect_model1.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred1)

In [ ]:
report = classification_report(y_test, y_pred1) 
print(report)

In [ ]:
cm = confusion_matrix(y_test, y_pred1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix');

# TF IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500)
X = tv.fit_transform(corpus).toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
spam_detect_model2 = RandomForestClassifier().fit(X_train, y_train)
y_pred2 = spam_detect_model2.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred2)

In [ ]:
report = classification_report(y_test, y_pred2) 
print(report)

In [ ]:
cm = confusion_matrix(y_test, y_pred2)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix');

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

**Iterating through each word**

In [ ]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [ ]:
import gensim

# Gensim word2vec model

In [ ]:
model=gensim.models.Word2Vec(words,window=5,min_count=2)

In [ ]:
#all the words
#model.wv.index_to_key

In [ ]:
#number of sentences in the corpus that was used to train the model.
model.corpus_count

In [ ]:
#one word is represented by 100 vectors
model.wv['work']

**Mean of vector elements in a sentence**

In [ ]:
#creating avg word2vec model for each sentence, all 100 vectors will be averaged
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

**Creating a list of avg sum of vectors in a sentence**

In [ ]:
X=[]
for i in range(len(words)):
    X.append(avg_word2vec(words[i]))

In [ ]:
type(X)

In [ ]:
X = np.array(X, dtype=object)

**# Actual length is 5572, but length of X is 5564, because stemming and stopwords removed some senttences, we will find them below.**

In [ ]:
[[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, df['Message']) if i<1]

**Removed y values where x is nan**

In [ ]:
y = df[list(map(lambda x: len(x)>0 ,corpus))]
y=pd.get_dummies(y['output_spam'])
y=y.iloc[:,1].values

y.shape

# Creating a new Dataframe for word2vec

In [ ]:
df1 = pd.DataFrame(X)

In [ ]:
df1['output'] = y

In [ ]:
df1.rename(columns = {0 : 'vector'}, inplace = True)

In [ ]:
df1['output'].replace({True : 1, False : 0}, inplace = True)

In [ ]:
df1.head()

**Removing Nan rows**

In [ ]:
df_nan_rows = df1[df1.isna().any(axis=1)]
print(df_nan_rows)

In [ ]:
df1.dropna(inplace = True)

In [ ]:
df1.shape

**Converting array sequence to list to fit in model**

In [ ]:
df1['vector'] = df1['vector'].apply(lambda x: list(x))

In [ ]:
X = df1['vector'].tolist()

In [ ]:
y= df1['output']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

In [ ]:
y_pred3 = model.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred3)

In [ ]:
report = classification_report(y_test, y_pred3) 
print(report)

In [ ]:
cm = confusion_matrix(y_test, y_pred3)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix');

# Word Embedding Techniques using Embedding Layer in Keras

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
len(corpus)

In [ ]:
### Vocabulary size
voc_size=10000

In [ ]:
#this will give list with index of words from voc_size, i.e if a sentence has 3 words, it will give list of those 3 word's indexes
AttributeError=[one_hot(words,voc_size)for words in corpus] 

**Model building**

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [ ]:
len(max(corpus))

In [ ]:
## pre padding
sent_length=75
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
## 100 feature dimesnions
dim=100

In [ ]:
from keras.layers import Embedding, Flatten

In [ ]:
model=Sequential()
model.add(Embedding(voc_size,1,input_length=sent_length))
model.add(Flatten())
model.compile('adam','mse')

In [ ]:
model.summary()

In [ ]:
x = model.predict(embedded_docs)

In [ ]:
y= df['output_spam']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [ ]:
spam_detect_model5 = RandomForestClassifier().fit(x_train, y_train)
y_pred5 = spam_detect_model5.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred5)

In [ ]:
cm = confusion_matrix(y_test, y_pred5)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix');

In [ ]:
report = classification_report(y_test, y_pred5)
print(report)